In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

# OpenAI Assistants API를 활용한 서비스 구현 튜토리얼 (2025년 3월 기준)

본 튜토리얼에서는 OpenAI Assistants API를 활용하여 AI 어시스턴트 기반 서비스를 개발하는 방법을 단계별로 설명합니다. 2025년 3월 기준의 OpenAI 공식 API 문서를 기반으로 최신 기능과 모범 사례를 반영했으며, Python 예제 코드를 통해 실습 형태로 진행됩니다. 이 튜토리얼은 ChatGPT 등의 언어 모델 API 사용 경험이 있고, 새로운 Assistants API 기능에 관심이 있는 개발자를 대상으로 합니다.

## 1. OpenAI Assistants API 소개

OpenAI Assistants API는 OpenAI의 새로운 API로, 대화형 AI 어시스턴트를 보다 쉽게 구축하고 관리할 수 있도록 설계되었습니다. 기존의 Chat Completion API가 무상태(stateless) 방식이어서 매 요청마다 대화 기록을 모두 보내야 했다면, Assistants API는 상태를 유지(stateful) 하는 대화 스레드(Thread)를 제공하여 자동으로 대화 컨텍스트를 관리합니다. 또한 함수 호출, 코드 실행, 지식 검색 등의 도구(Tools) 를 어시스턴트에 병렬로 연결할 수 있어 훨씬 강력한 코파일럿(copilot) 형태의 서비스를 구축할 수 있습니다. 

왜 Assistants API를 사용해야 할까요? 몇 가지 주요 장점을 정리하면 다음과 같습니다:

- 대화 지속성 & 메모리: Assistants API는 대화 스레드에 메시지 내역을 지속적으로 저장하며, 컨텍스트 윈도우 제한 내에서 자동으로 요약/잘라내기를 수행합니다. 개발자는 별도의 상태 관리 로직 없이 장기간의 대화도 처리할 수 있습니다.

- 다양한 도구 통합: 어시스턴트가 코드 실행(Code Interpreter), 지식 검색(File Retrieval), 함수 호출(Function Calling) 등의 도구를 직접 사용하도록 구성할 수 있습니다. 이를 통해 모델의 한계를 넘어서는 작업(예: 데이터베이스 질의, 파일 처리, 외부 API 호출 등)을 자동화할 수 있습니다.

- 개인화된 지시어: 어시스턴트를 생성할 때 지시어(Instructions) 를 주입함으로써 해당 어시스턴트의 성격과 역할을 미리 정의할 수 있습니다. 예를 들어 “당신은 친절한 고객지원 봇입니다”와 같은 시스템 지시어로 어시스턴트의 톤과 도메인 지식을 설정할 수 있습니다.

- 모델 기능 향상: 2023년 11월 이후 공개된 GPT-3.5/4 모델 (예: gpt-3.5-turbo-1106, gpt-4-1106-preview)은 함수 호출과 같은 고급 기능을 지원하며 Assistants API와 연동됩니다. Assistants API를 사용하면 이러한 최신 모델 기능을 간편하게 활용할 수 있습니다.

- 개발 편의: OpenAI 플랫폼의 웹 인터페이스(플레이그라운드)뿐 아니라 API를 통해서도 동일한 어시스턴트를 생성/관리할 수 있습니다. 한 번 만든 어시스턴트를 재사용하거나 여러 쓰레드를 동시에 운영하는 것이 쉬워집니다. (예: 하나의 어시스턴트에 대해 다수의 사용자 대화 스레드를 관리)

요약하면, Assistants API는 기존 ChatGPT API의 상태 관리 진화판으로 볼 수 있습니다. 복잡한 대화 상태 관리, 외부 도구 통합, 문서 임베딩 등 많은 부분을 OpenAI 플랫폼이 맡아주므로, 개발자는 핵심 로직 구현에 집중할 수 있습니다. 이러한 이유로, 코파일럿 스타일의 챗봇이나 자동화 에이전트를 만든다면 Assistants API가 강력한 선택지가 됩니다.

>Note: Assistants API는 2025년 초 현재 베타(beta) 단계로, OpenAI가 지속적으로 기능을 개선하고 있습니다. 사용 전에 최신 문서를 확인하고, 베타 API인 만큼 일부 동작이나 요금 정책이 변경될 수 있음을 유의하세요.

## 2. API 키 설정 및 환경 변수 사용

OpenAI API를 사용하려면 API 키가 필요합니다. OpenAI 플랫폼의 API 키 관리 페이지에서 비밀 키를 생성할 수 있습니다. 생성된 키는 한 번만 표시되므로, 반드시 복사하여 안전한 곳에 저장하세요. 일반적으로 이 키를 소스 코드에 하드코딩하지 않고, 별도의 설정으로 관리하는 것이 좋습니다. **환경 변수(Environment Variable)**를 사용하면 API 키를 소스 코드에 노출하지 않고 관리할 수 있습니다. 개발 PC 또는 서버의 환경 변수 OPENAI_API_KEY에 키를 저장해 두면, OpenAI 라이브러리가 자동으로 이를 읽어 사용할 수 있습니다. Python 개발 환경에서는 python-dotenv 패키지를 활용해 .env 파일에 키를 저장하고 로드하는 방식이 편리합니다. 

다음은 API 키를 설정하고 로드하는 과정입니다:

1. python-dotenv 설치: 터미널에서 pip install python-dotenv 명령으로 설치합니다 (한번만 수행).

2. 환경 변수 파일 생성: 프로젝트 루트 디렉토리에 .env 파일을 만들고, 아래와 같이 OpenAI API 키를 입력합니다 (따옴표 없이 실제 키로 대체).

    ```
    OPENAI_API_KEY=sk-***********************
    ```
    
3. 코드에서 로드: Python 코드에서 python-dotenv를 이용해 .env를 로드하고, os.environ을 통해 키를 불러옵니다. OpenAI 공식 Python SDK에서는 환경 변수 OPENAI_API_KEY를 자동으로 인식하므로, 명시적으로 지정하지 않아도 동작합니다. 그래도 명확성을 위해 아래와 같이 작성할 수 있습니다:

In [21]:
# 6월 업그레이드 경고 무시
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
import os

# 1. OpenAi 클라언트 생성
# print(os.getcwd())
load_dotenv(".env") # 환경변수 load
api_key = os.getenv("OPENAI_API_KEY")
print(api_key[:10], len(api_key))

client = OpenAI(api_key=api_key)

sk-proj-ik 164


In [10]:
# 단순 요청
# response = client.chat.completions.create(
#     model="gpt-4.1-nano",
#     messages="지금 서울 날씨는?"
# )
# assistant API이용하여 (1)대화이력을 저장 (2)파일첨부 (3)함수호출

## 3. 기본적인 Assistant 생성 및 메시지 보내기

가장 먼저, **어시스턴트(Assistant)**를 하나 만들어보겠습니다. 어시스턴트는 대화를 수행하는 주체로, 사용자의 요청을 받아 처리하는 AI 에이전트라고 볼 수 있습니다. Assistants API에서 어시스턴트를 생성할 때는 어떤 모델을 사용할지, 어떤 지시어(Instructions) 를 가질지, 그리고 사용할 **툴(Tools)**이 무엇인지 등을 설정할 수 있습니다. 예를 들어 간단한 챗봇 어시스턴트를 만들어 사용자 질문에 답변해보겠습니다.

In [11]:
# 2. 어시스턴트 생성
assistant = client.beta.assistants.create(
    name = "HelpBot",
    instructions= "당신은 유능하고, 친절한 도움말 어시스턴트입니다. 사용자 질문에 20글자 이내로 친절하게 답변하세요",
    model = "gpt-4o-mini",
    # tools 매개변수를 생략하거나, 빈리스트로 두면 기본 대화만 수행(저장)
)

In [14]:
assistant
print('llm에 요청시 필요한 assistant.id=', assistant.id)

llm에 요청시 필요한 assistant.id= asst_zhkDJtXbXx9YiBhvnRsqGVii


위 코드에서는 client.beta.assistants.create 메서드를 사용해 새로운 어시스턴트를 생성했습니다. 주요 파라미터를 살펴보면:

- name: 어시스턴트의 이름을 지정합니다. 콘솔이나 리스트에서 구분하기 위한 용도이며, 모델 응답 내용에는 영향을 미치지 않습니다.

- instructions: 시스템 레벨의 지시어로, 해당 어시스턴트가 모든 대화에서 따르게 될 기본 규칙이나 역할을 정의합니다. 여기서는 "친절한 도움말 어시스턴트"라는 성격을 부여했습니다. (이 지시어는 기존 ChatGPT의 시스템 메시지와 유사한 역할입니다.)

- model: 어시스턴트가 사용할 언어 모델을 지정합니다. 최신 기능을 쓰려면 OpenAI가 Nov 2023 이후 출시한 모델(-1106가 붙은 모델명)을 권장합니다. 


- assistant 생성을 제공하는 모델
```
 GPT-4 계열:
    gpt-4 - 기본 GPT-4 모델
    gpt-4-turbo - 더 빠르고 효율적인 GPT-4
    gpt-4o - 최신 멀티모달 모델
    gpt-4o-mini - 경량화된 GPT-4o

 GPT-3.5 계열:
    gpt-3.5-turbo - 빠르고 효율적인 모델(추천)

 기타:
    dall-e-3 - 이미지 생성용
    whisper-1 - 음성 인식용
    tts-1, tts-1-hd - 텍스트 음성 변환용
```

- tools: 어시스턴트에 활성화할 도구 목록입니다. 기본적인 Q&A 챗봇에서는 특별한 툴이 필요 없으므로 비워두었습니다. (tools=[] 또는 생략) 나중에 코드 인터프리터나 함수 호출 등을 사용하게 될 경우 이 필드를 설정합니다.

어시스턴트가 성공적으로 생성되면 고유 ID (assistant.id)가 반환됩니다. 이 ID는 이후 대화 스레드에서 어떤 어시스턴트를 사용할지 지정할 때 필요하므로 저장해둡니다. (참고로, client.beta.assistants.list()를 호출하면 계정 내 모든 어시스턴트 목록을 확인할 수 있습니다.) 

이제 이 어시스턴트와 대화를 시작해보겠습니다. 대화를 하기 위해서는 **스레드(Thread)**를 생성해야 합니다. 스레드는 개별적인 대화 세션을 나타내며, 사용자와 어시스턴트 간의 메시지(Message) 목록을 포함합니다. 하나의 어시스턴트는 여러 개의 스레드를 가질 수 있는데, 예를 들어 같은 어시스턴트를 여러 사용자가 동시에 이용하면 각 사용자별로 별도 스레드가 생깁니다. 

스레드를 만들고, 그 안에 사용자 메시지를 추가한 뒤, 어시스턴트의 답변을 요청하는 일련의 과정을 코드로 수행해봅시다:


In [36]:
# 3. 새로운 대화 스레드 생성 : 기억담당 (처음에는 메시지 없음)
thread = client.beta.threads.create()
# print('llm에 요청할 때 필요한 id :', thread.id)

# 4. 스레드에 사용자 프롬프트 추가
#user_message = 
client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "오늘 날씨 몇도까지 올라가요?", # 사용자 프롬프트
)

# 5. llm 요청(어시스턴트에게 답변 생성 요청) -> 답변내용이 thread에 자동 추가
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)
print(run)

Run(id='run_HgJxFURnTAj0jbmY10Pv4O7E', assistant_id='asst_zhkDJtXbXx9YiBhvnRsqGVii', cancelled_at=None, completed_at=1751250118, created_at=1751250115, expires_at=None, failed_at=None, incomplete_details=None, instructions='당신은 유능하고, 친절한 도움말 어시스턴트입니다. 사용자 질문에 20글자 이내로 친절하게 답변하세요', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1751250117, status='completed', thread_id='thread_fBfzVYLdyBWYsJ3pdTen06W5', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=15, prompt_tokens=74, total_tokens=89, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)


In [58]:
# 6. 실행 완료후, 스레드의 모든 메세지 불러오기
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages.data
# 6-1. 메세지를 시간순서대로 정렬
sorted_messages = sorted(messages.data,  # 정렬 list
                         key=lambda data : data.created_at) # 정렬 기준 
# 7. thread에 저장된 메세지들 출력
#print(sorted_messages)

for msg in sorted_messages:    
    #print("{}({}) : {}".format(msg.role, msg.created_at, msg.content[0].text.value))
    dateStyle = time.localtime(msg.created_at)
    dateStyle_str = time.strftime("%Y-%m-%d %H:%M:%S", dateStyle)
    print("{:9}({}) : {}".format(msg.role, dateStyle_str, msg.content[0].text.value))

user     (2025-06-30 11:21:54) : 오늘 날씨 몇도까지 올라가요?
assistant(2025-06-30 11:21:57) : 현재 위치에 따라 다릅니다! 확인해보세요.


In [22]:
# list 정렬
students = ['홍', '김', '이']
# students.sort() # students list(원본)를 정렬시킴
sorted_students = sorted(students) # 원본은 그대로 정렬본(sorted_students)은 따로 생성
print("원본 :", students)
print("정렬본 :", sorted_students)

원본 : ['홍', '김', '이']
정렬본 : ['김', '이', '홍']


In [32]:
import time
students = [
    {'content':'홍길동임다', 'created_at':time.time(), 'score':95},
    {'content':'김길동임다', 'created_at':time.time()+60*60, 'score':90},
    {'content':'이길동임다', 'created_at':1751248564, 'score':100},
]
sorted_students = sorted(students,
                        key=lambda data : data['score'], # 정렬기준
                        reverse=True) # 내림차순
print('정렬본(score기준 내림차순) :', sorted_students)
# 원본은 created_at기준 오름차순 정렬을 적용
students.sort(key=lambda data : data['created_at'])
print('정렬된 students :', students)

정렬본(score기준 내림차순) : [{'content': '이길동임다', 'created_at': 1751248564, 'score': 100}, {'content': '홍길동임다', 'created_at': 1751249740.6286507, 'score': 95}, {'content': '김길동임다', 'created_at': 1751253340.6286507, 'score': 90}]
정렬된 students : [{'content': '이길동임다', 'created_at': 1751248564, 'score': 100}, {'content': '홍길동임다', 'created_at': 1751249740.6286507, 'score': 95}, {'content': '김길동임다', 'created_at': 1751253340.6286507, 'score': 90}]


위 코드 설명:

- client.beta.threads.create(): 새로운 대화 스레드를 생성합니다. 반환된 thread 객체는 고유 ID (thread.id)를 갖습니다.

- client.beta.threads.messages.create(...): 특정 스레드에 새로운 메시지를 추가합니다. role 파라미터로 이 메시지의 주체를 지정하는데, 사용자 메시지이므로 "user"로 설정하고 content에 사용자의 질문 내용을 넣습니다.

- client.beta.threads.runs.create_and_poll(...): 해당 스레드를 주어진 어시스턴트로 실행(run)합니다. assistant_id를 우리가 생성한 어시스턴트의 ID로 지정해야 합니다. 여기서는 create_and_poll를 사용하여 실행이 완료될 때까지 블록합니다. (이 함수는 내부적으로 주기적으로 run.status를 체크하여 완료될 때까지 기다립니다.)

    >참고: create_and_poll를 사용하지 않고 runs.create를 쓰는 경우, 즉시 run 객체를 받고 수동으로 상태를 확인해야 합니다. 간단한 예에서는 편의를 위해 create_and_poll로 한 번에 처리했습니다.

- 대화 실행이 완료되면 (run.status == "completed") 해당 스레드에는 사용자 질문과 어시스턴트 답변, 총 두 개의 메시지가 들어있게 됩니다. 이를 threads.messages.list로 가져와서 출력했습니다.

어시스턴트가 우리의 지시어에 따라 친절한 톤으로 답변한 것을 볼 수 있습니다. 이처럼 스레드 생성 -> 메시지 추가 -> 실행 -> 결과 조회의 흐름으로 어시스턴트와의 기본 대화를 구현할 수 있습니다.

>🔍 참고: thread는 대화의 흐름(메시지 목록)을 나타내고, run은 어시스턴트가 해당 스레드의 마지막 사용자 메시지에 응답하는 실행 과정을 가리킵니다. 일반적으로 하나의 사용자 메시지에 대해 하나의 run이 생성되며, 어시스턴트가 답변을 완성하면 run.status가 "completed"로 바뀝니다. (코드에서는 create_and_poll를 사용해 이 과정을 추상화했습니다.)

이제 기본적인 Q&A 챗봇 형태의 어시스턴트를 만들고, 질문에 답변을 받는 방법을 확인했습니다. 다음으로는 파일을 어시스턴트에 활용하는 방법과 코드 실행 도구를 사용하는 방법을 살펴보겠습니다.


### Assistant_chatbot (assistant_chatbot.py)
```
1. client 생성
2. assistant 생성 
3. thread 생성 : 기억담당
while True:
    4. thread에 user 메세지 추가
    5. run 수행(assistant.id, thread.id필요)
    6. 최종 assistant 답변 출력
7. 대화 이력 뽑기
```

In [3]:
import time
import os
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
import warnings
warnings.filterwarnings("ignore")
# 1. client 생성
load_dotenv(".env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
# 2. assistant 생성 
assistant_cs = client.beta.assistants.create(
    name="CustomerSupportBot",
    instructions="당신은 고객 지원 쳇봇입니다. 사용자 문의에 대해 30자이내로 친절한 답변을 하세요",
    model = "gpt-4o-mini"
)
# 3. thread 생성 : 기억담당
thread_cs = client.beta.threads.create()
print("쳇봇이 시작됩니다(종료:종료나 exit). 모든 대화이력은 저장됩니다")

while True:
    user_input = input("User :").strip()
    if user_input.lower() in ('종료', 'exit'):
        print('쳇봇이 종료됩니다. 이용해서 주셔서 감사합니다')
        break
    if user_input == "":
        continue
    # 4~6 : user_input을 thread_cs에 추가하고 실행한후 최종 답변 출력
    # 4. 스레드에 user_input을 추가
    client.beta.threads.messages.create(
        thread_id= thread_cs.id,
        role     = "user",
        content  = user_input
    )
    # 5. 실행
    client.beta.threads.runs.create_and_poll(
        thread_id   =thread_cs.id,
        assistant_id=assistant_cs.id
    )
    # 6. 최종 답변 출력
    messages = client.beta.threads.messages.list(thread_id=thread_cs.id)
    assistant_reply = messages.data[0]
    reply_text = assistant_reply.content[0].text.value
    # print(f"user : {user_input}")
    print(f"assistant : {reply_text}")

쳇봇이 시작됩니다(종료:종료나 exit). 모든 대화이력은 저장됩니다
User :2006년생 음주 가능해?
assistant : 2024년이면 가능합니다.
User :주류 추천해줘
assistant : 달콤한 와인이나 칵테일 어때요?
User :종료
쳇봇이 종료됩니다. 이용해서 주셔서 감사합니다


In [4]:
# 7. 대화 이력 뽑아, 파일 출력(content[0].text.value)
sored_messages = sorted(messages.data,
                       key=lambda msg : msg.created_at)
with open('data/ch7_chat_history.txt', 'w', encoding='utf-8') as f:
    for message in sored_messages:
        # 생성 시각(message.created_at)을 datetime으로 변환
        datetime_info = time.localtime(message.created_at)
        # 보기 좋은 문자열 형식으로 변환
        output_str     = time.strftime("%y-%m-%d %H:%M:%S", datetime_info)
        # 파일 출력
        f.write("{:9}({}) : {}\n".format(message.role,
                                        output_str,
                                        message.content[0].text.value))

## 4. 파일 업로드 및 코드 인터프리터 활용

OpenAI Assistants API에서 파일 업로드는 사용자가 제공한 데이터를 어시스턴트가 활용할 수 있게 하는 중요한 기능입니다. 예를 들어 PDF 문서를 요약하거나, CSV 데이터를 분석하거나, 이미지를 처리하는 등의 작업이 가능해집니다. Assistants API에서는 파일을 미리 업로드하고 file ID를 통해 어시스턴트와 대화 스레드에 첨부하는 방식을 사용합니다. 이렇게 첨부된 파일들은 어시스턴트의 도구를 통해 접근할 수 있습니다. 

파일을 다루는 대표적인 도구로 **코드 인터프리터(Code Interpreter)**가 있습니다. 코드 인터프리터를 활성화하면 어시스턴트는 Python 코드를 작성하고 실행하여 결과를 산출할 수 있게 됩니다. 이때 외부 파일이 필요하면, 사전에 업로드된 파일을 코드에서 읽거나 쓸 수 있습니다. 이제 파일 업로드와 코드 인터프리터 사용 예제를 통해 이를 이해해보겠습니다. 

먼저 파일 업로드 절차입니다. OpenAI Python SDK에서는 client.files.create 메서드를 제공하며, 여기에 파일 객체와 용도를 지정하여 업로드할 수 있습니다. 파일 용도는 purpose='assistants'로 설정해야 Assistants API 대화에서 활용 가능한 리소스로 저장됩니다. 한 번 업로드된 파일은 OpenAI 서버에 저장되며, 반환되는 file.id를 이용해 어시스턴트나 스레드에 첨부할 수 있습니다.


In [12]:
# 2. 코드 인터프리터(Code Interpreter) 기능을 갖춘 어시스턴트 생성
assistant = client.beta.assistants.create(
    name="DataAnalyzer",
    instructions="당신은 데이터 분석을 돕는 어시스턴트입니다. Python 코드를 작성하여 사용자 요청을 해결하세요",
    model = "gpt-4o-mini",
    tools=[{
        "type":"code_interpreter" # 코드 인터프리터를 활성화
    }]
)

위 예에서는 현재 디렉토리에 있는 data.csv 파일을 업로드했습니다. purpose를 'assistants'로 지정했으므로 이 파일은 어시스턴트의 Retrieval(지식 검색) 툴이나 Code Interpreter에서 사용할 수 있는 상태가 됩니다. 업로드 성공 시 file_obj.id에 할당된 값 (예: file-abc123...)을 확보합니다. 이 ID는 마치 파일 핸들처럼 사용되어, 이후 어시스턴트나 메시지에 파일을 연결할 때 쓰입니다.

이제 **코드 인터프리터(Code Interpreter)**를 사용하도록 어시스턴트를 업데이트해보겠습니다. 기존에 만든 HelpBot 어시스턴트에 코드 인터프리터 툴을 추가할 수도 있지만, 튜토리얼의 연속성을 위해 새로운 어시스턴트를 생성해보겠습니다.

여기서 tools에 {"type": "code_interpreter"}를 추가하여 코드 실행 기능을 켰습니다. GPT-4 모델을 선택한 것은 코드 해석 및 작성 능력이 더 뛰어나기 때문입니다. (GPT-3.5로도 코드 인터프리터를 사용할 수 있으나 복잡한 코드는 GPT-4가 더 정확하게 수행합니다.) 

- OpenAI Assistants API에서 사용 가능한 주요 모델들은 다음과 같습니다

> GPT-4 모델군
gpt-4o - 최신 GPT-4 Omni 모델 (텍스트, 이미지, 오디오 처리 가능)
gpt-4o-mini - GPT-4 Omni의 경량화 버전 (비용 효율적)
gpt-4-turbo - GPT-4 Turbo 모델
gpt-4 - 기본 GPT-4 모델

>GPT-3.5 모델군
gpt-3.5-turbo - GPT-3.5 Turbo 모델

>코드 인터프리터 사용 시 권장 모델
코드 인터프리터를 사용하는 데이터 분석 작업의 경우:
gpt-4o - 가장 강력한 성능, 복잡한 분석 작업에 최적
gpt-4o-mini - 비용 대비 성능이 우수, 일반적인 데이터 분석에 적합
gpt-4-turbo - 안정적인 성능, 대용량 데이터 처리에 좋음


이제 이 어시스턴트에게 우리가 업로드한 data.csv 파일을 이용한 작업을 요청해보겠습니다. 예를 들어, CSV에 숫자 목록이 들어있다고 가정하고 그 합계를 계산하도록 해봅시다. 사용자는 파일을 첨부하며 질문을 던지고, 어시스턴트는 코드 인터프리터로 파일을 읽어 합계를 계산한 후 답을 돌려줄 것입니다.


In [22]:
# 3. 파일 업로드 생성
file_obj = client.files.create(
    file=open('data/data.csv', 'rb'),
    purpose="assistants"
)
print('쓰레드에 message 추가시 필요한 id :', file_obj.id)

쓰레드에 message 추가시 필요한 id : file-Hd9z6j3iWeLUDRoUAeCBEn


In [29]:
# 4. 스레드 시작 -> 스레드에 사용자 메세지 추가(파일 첨부하여 질문)
thread = client.beta.threads.create()
question = "첨부된 파일의 숫자들를 더하는 python 코드를 작성하고 실행한 뒤, 코드와 결과를 모두 보여주세요"
client.beta.threads.messages.create(
    thread_id= thread.id,
    content  = question,
    role     = "user",
    attachments=[{
        "file_id" : file_obj.id,
        "tools": [{"type":"code_interpreter"}] 
    }]
)

Message(id='msg_mE8yNUGlksRM2GhdQ6sAsq29', assistant_id=None, attachments=[Attachment(file_id='file-Hd9z6j3iWeLUDRoUAeCBEn', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='첨부된 파일의 숫자들를 더하는 python 코드를 작성하고 실행한 뒤, 코드와 결과를 모두 보여주세요'), type='text')], created_at=1751265320, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_6QvS0g1LTJWO7ZEyKy7UgJZb')

In [30]:
# 5. 어시스턴트 실행 (llm 요청)
client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 6. 실행한 후, 어시스턴트 답변확인
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [31]:
assistant_reply = messages.data[0]
result = assistant_reply.content[0].text.value
print(result)

다음은 파일에서 숫자를 읽고 더한 Python 코드입니다:

```python
# 파일 경로
file_path = '/mnt/data/file-Hd9z6j3iWeLUDRoUAeCBEn'

# 파일의 내용을 읽어 숫자를 추출하여 더하는 코드
with open(file_path, 'r') as file:
    # 파일 내용을 읽습니다.
    contents = file.read()
    # 숫자 추출
    numbers = [int(num) for num in contents.split() if num.isdigit()]
    # 숫자들의 합 계산
    total_sum = sum(numbers)

total_sum, numbers
```

실행 결과:
- 파일에서 추출된 숫자: `[10, 20, 30, 40, 50]`
- 합계: `150` 

총합은 150입니다.


위 코드에서 특별히 눈여겨볼 부분은 사용자 메시지를 생성할 때 attachments=[{"file_id": file_id, "tools": [{"type": "code_interpreter"}]}] 로 파일을 첨부한 것입니다. 이렇게 하면 OpenAI가 해당 스레드 컨텍스트에 해당 파일을 연계시켜 줍니다. 코드 인터프리터는 이 파일을 가상 환경의 루트 경로에 다운받아놓고, Python에서 open("파일명")으로 내용을 읽을 수 있도록 해줍니다. (참고로, 파일명은 업로드된 원래 이름인 data.csv 그대로 사용 가능합니다.) 

어시스턴트는 질문을 이해하고, 코드 인터프리터 툴을 통해 data.csv 파일을 열어 합계를 계산하는 Python 코드를 실행할 것입니다. 실행 결과는 어시스턴트의 답변으로 반환됩니다.

이처럼 코드 인터프리터를 활용하면 어시스턴트가 사용자 제공 데이터를 직접 분석하거나 처리한 후 결과를 알려줄 수 있습니다. ChatGPT의 Code Interpreter 기능을 API로 사용할 수 있게 된 셈입니다. 이미지 처리, 데이터 시각화, 파일 변환 등 다양한 작업을 자동화할 수 있으므로 활용 범위가 매우 넓습니다.

>Tips: 코드 인터프리터 사용 시 고려사항:
- 실행 시간: 코드 실행에는 시간이 걸릴 수 있습니다. runs.create_and_poll를 쓰면 자동으로 대기해주지만, 복잡한 작업의 경우 타임아웃이나 지연을 염두에 두어야 합니다.

- 파일 접근: 첨부된 파일들은 코드 인터프리터 세션 내에서 읽기/쓰기 가능합니다. 어시스턴트가 새로운 파일(예: 그래프 이미지)을 생성하면, 그 파일도 자동으로 OpenAI 파일 스토리지에 업로드되고 ID가 부여됩니다. 어시스턴트의 답변에는 해당 파일 ID가 인용 형태로 포함될 수 있습니다. (예: “보고서 결과는 파일 file_xyz에 저장되었습니다.”) 이러한 파일은 client.files.list() 등을 통해 확인할 수 있습니다.

- 보안: 코드 인터프리터는 격리된 샌드박스 환경에서 동작하므로 어시스턴트가 인터넷에 직접 접속하거나 로컬 시스템에 영향을 주지 않습니다. 다만, 악성 코드나 무한 루프 등으로 오남용되지 않도록 OpenAI 측에서 실행 시간을 제한하고 있음을 참고하세요.


### Quiz : data/ch06_quiz.txt을 첨부한 후 파일의 내용을 30자로 요약한 결과 출력
```
1. client 생성(윗 client 사용)
2. file첨부 객체
3. assistant 생성 (code_interpreter tools 추가)
4. thread 시작
5. thread에 메세지 추가(첨부파일)
6. 어시스턴트 실행
7. 어시스턴트 reply 메세지 추출
```

In [32]:
# 1. client 생성(윗 client 사용)
# 2. file첨부 객체
file_obj = client.files.create(
    file=open('data/ch06_quiz.txt', 'rb'),
    purpose='assistants'
)
# 3. assistant 생성 (code_interpreter tools 추가)
assistant = client.beta.assistants.create(
    name="DataAnalyzer",
    instructions="당신은 훌륭한 어시스턴트예요. 필요시 Python 코드를 작성하여 사용자 요청을 해결하세요",
    model = "gpt-4o-mini",
    tools=[{
        "type":"code_interpreter" # 코드 인터프리터를 활성화
    }]
)
# 4. thread 시작
thread = client.beta.threads.create()

# 5. thread에 메세지 추가(첨부파일)
question = ("첨부파일을 읽고 2줄로 글번호를 달아 요약하는 python 코드를 작성하여 "
            "실행한 뒤 그 결과를 보여주세요")
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=question,
    attachments=[{
        "file_id":file_obj.id,
        "tools" : [{"type":"code_interpreter"}]
    }]
)
# 6. 어시스턴트 실행
client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 7. 실행 후 답변 확인
messages = client.beta.threads.messages.list(thread_id=thread.id)

print(messages.data[0].content[0].text.value)

파일 내용을 30글자 내외로 요약한 결과는 다음과 같습니다:

1: 인공지능(AI)은 인간이나 동물의 자연 지능과 달리 기...
2: 주요 AI 교재에서는 이를 "지능형 에이전트"의 연구로...
3: AI의 대표적인 응용 분야로는 자연어 처리(NL...
4: 예를 들어, 자연어 처리는 사람의 언어를 이해하고 생성...
5: 컴퓨터 비전은 이미지를 분석하고 이해하는 기술로, 얼굴...
6: AI 기술의 발전은 산업과 일상생활에 큰 변화를...

추가로 도움이 필요하시면 말씀해 주세요!
